In [1]:
import os
import requests
import json
from datetime import datetime
import boto3
from botocore.exceptions import ClientError

In [ ]:
from genson import SchemaBuilder
import json

API_KEY = "cf4349d9377ef4a58c895a306e2cc50e6a011882e45e32dd07d0adce80b13a5b"
API_URL = "https://api.openaq.org/v3/sensors/235/measurements"


all_measurements = []
headers = {"X-API-Key": API_KEY}

response = requests.get(API_URL, headers=headers)
response.raise_for_status()  # Raise HTTP errors

data = response.json()

pretty_json = json.dumps(data, indent=5)
print(pretty_json)

builder = SchemaBuilder()
builder.add_object(data)  # `data` is your JSON object

schema = builder.to_schema()
print(json.dumps(schema, indent=4))


{
     "meta": {
          "name": "openaq-api",
          "website": "/",
          "page": 1,
          "limit": 100,
          "found": ">100"
     },
     "results": [
          {
               "value": 5.0,
               "flagInfo": {
                    "hasFlags": false
               },
               "parameter": {
                    "id": 2,
                    "name": "pm25",
                    "units": "\u00b5g/m\u00b3",
                    "displayName": null
               },
               "period": {
                    "label": "raw",
                    "interval": "01:00:00",
                    "datetimeFrom": {
                         "utc": "2016-01-30T00:00:00Z",
                         "local": "2016-01-30T00:00:00Z"
                    },
                    "datetimeTo": {
                         "utc": "2016-01-30T01:00:00Z",
                         "local": "2016-01-30T01:00:00Z"
                    }
               },
               "coordinates": n

In [12]:
import requests
import boto3
import time
from botocore.exceptions import ClientError

# Configuration
S3_BUCKET = "env-raw"
API_BASE = "https://api.openaq.org/v3"
REQUEST_DELAY = 1
API_KEY = "cf4349d9377ef4a58c895a306e2cc50e6a011882e45e32dd07d0adce80b13a5b"
headers = {"X-API-Key": API_KEY}

s3 = boto3.client('s3')

def fetch_paginated(endpoint, params=None):
    """Fetch all pages from a paginated OpenAQ endpoint"""
    page = 1
    all_results = []
    
    while True:
        response = requests.get(
            f"{API_BASE}/{endpoint}",
            headers=headers,
            params={"page": page, "limit": 1000, **(params or {})}
        )
        response.raise_for_status()
        data = response.json()
        
        # Store raw response in S3
        #s3_key = f"openaq/{endpoint}/page_{page}.json"
        #s3.put_object(Bucket=S3_BUCKET, Key=s3_key, Body=response.text)
        
        all_results.extend(data['results'])
        
        # Handle pagination for string values like ">100"
        found = data['meta']['found']
        limit = data['meta']['limit']
        
        try:
            # Case 1: "found" is an integer
            found_int = int(found)
            if page * limit >= found_int:
                break
        except ValueError:
            # Case 2: "found" is a string (e.g., ">100")
            # Break if current page has fewer results than limit
            if len(data['results']) < limit:
                break
        
        page += 1
        time.sleep(REQUEST_DELAY)
    
    return all_results

def main():
    # 1. Get all locations
    locations = fetch_paginated("locations")
    
    # 2. Get sensors for each location
    for location in locations:
        location_id = location['id']
        sensors = fetch_paginated(f"locations/{location_id}/sensors")
        
        # 3. Get measurements for each sensor
        for sensor in sensors:
            sensor_id = sensor['id']
            fetch_paginated(f"sensors/{sensor_id}/measurements", params={
                "sort": "desc",
                "date_from": "2000-01-01",
                "date_to": "2024-12-31"
            })

if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        print(f"Script failed: {str(e)}")
        raise

Script failed: ('Connection broken: IncompleteRead(527178 bytes read, 756854 more expected)', IncompleteRead(527178 bytes read, 756854 more expected))


ChunkedEncodingError: ('Connection broken: IncompleteRead(527178 bytes read, 756854 more expected)', IncompleteRead(527178 bytes read, 756854 more expected))

In [ ]:
import requests
import json
import os
import time
from datetime import datetime

# Configuration
API_BASE = "https://api.openaq.org/v3"
REQUEST_DELAY = 1
API_KEY = "cf4349d9377ef4a58c895a306e2cc50e6a011882e45e32dd07d0adce80b13a5b"
headers = {"X-API-Key": API_KEY}
LOCAL_SAVE_DIR = "./data/bronze"  # Local storage directory

# Create save directory if not exists
os.makedirs(LOCAL_SAVE_DIR, exist_ok=True)

def save_local(data, endpoint, page=1):
    """Save API response to local file system"""
    # Create endpoint directory
    endpoint_dir = os.path.join(LOCAL_SAVE_DIR, endpoint)
    os.makedirs(endpoint_dir, exist_ok=True)
    
    # Create filename with timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"page_{page}_{timestamp}.json"
    filepath = os.path.join(endpoint_dir, filename)
    
    # Save as pretty-printed JSON
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2)
        
    return filepath

def fetch_paginated(endpoint, params=None):
    """Fetch all pages from a paginated OpenAQ endpoint"""
    page = 1
    all_results = []
    
    while True:
        try:
            response = requests.get(
                f"{API_BASE}/{endpoint}",
                headers=headers,
                params={"page": page, "limit": 1000, **(params or {})}
            )
            response.raise_for_status()
            data = response.json()
            
            # Save raw response locally
            save_local(data, endpoint, page)
            
            all_results.extend(data['results'])
            
            # Handle pagination
            found = data['meta']['found']
            limit = data['meta']['limit']
            
            try:
                found_int = int(found)
                if page * limit >= found_int:
                    break
            except ValueError:
                if len(data['results']) < limit:
                    break
            
            page += 1
            time.sleep(REQUEST_DELAY)
            
        except requests.exceptions.RequestException as e:
            print(f"Error fetching {endpoint} page {page}: {str(e)}")
            break
    
    return all_results

def main():
    print(f"Saving data to: {os.path.abspath(LOCAL_SAVE_DIR)}")
    
    # 1. Get all locations
    print("Fetching locations...")
    locations = fetch_paginated("locations")
    
    # 2. Get sensors for each location
    for idx, location in enumerate(locations, 1):
        location_id = location['id']
        print(f"Processing location {idx}/{len(locations)} (ID: {location_id})")
        
        sensors = fetch_paginated(f"locations/{location_id}/sensors")
        
        # 3. Get measurements for each sensor
        for sensor_idx, sensor in enumerate(sensors, 1):
            sensor_id = sensor['id']
            print(f"  Sensor {sensor_idx}/{len(sensors)} (ID: {sensor_id})")
            
            fetch_paginated(f"sensors/{sensor_id}/measurements", params={
                "sort": "desc",
                "date_from": "2000-01-01",
                "date_to": datetime.now().strftime("%Y-%m-%d")  # Dynamic end date
            })

if __name__ == "__main__":
    start_time = datetime.now()
    try:
        main()
        duration = datetime.now() - start_time
        print(f"Completed in {duration}")
    except Exception as e:
        print(f"Script failed: {str(e)}")
        raise

Saving data to: c:\Users\joaob\OneDrive\Área de Trabalho\Efrei\Data Lakes\Projet\openaq_data
Fetching locations...
Processing location 1/19767 (ID: 3)
  Sensor 1/2 (ID: 5)
  Sensor 2/2 (ID: 6)
Processing location 2/19767 (ID: 4)
  Sensor 1/2 (ID: 8)
  Sensor 2/2 (ID: 7)
Processing location 3/19767 (ID: 5)
  Sensor 1/2 (ID: 10)
  Sensor 2/2 (ID: 9)
Processing location 4/19767 (ID: 6)
  Sensor 1/2 (ID: 12)
  Sensor 2/2 (ID: 11)
Processing location 5/19767 (ID: 7)
  Sensor 1/2 (ID: 14)
  Sensor 2/2 (ID: 13)
Processing location 6/19767 (ID: 8)
  Sensor 1/2 (ID: 15)
  Sensor 2/2 (ID: 16)
Processing location 7/19767 (ID: 9)
  Sensor 1/2 (ID: 17)
  Sensor 2/2 (ID: 18)
Processing location 8/19767 (ID: 10)
  Sensor 1/2 (ID: 20)
  Sensor 2/2 (ID: 19)
Processing location 9/19767 (ID: 11)
  Sensor 1/2 (ID: 22)
  Sensor 2/2 (ID: 21)
Processing location 10/19767 (ID: 12)
  Sensor 1/1 (ID: 23)
Processing location 11/19767 (ID: 13)
  Sensor 1/3 (ID: 13866)
Error fetching sensors/13866/measurements pag

# Corrected BRONZE

In [ ]:
import requests
import json
import os
import time
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed

# Configuration
API_BASE = "https://api.openaq.org/v3"
REQUEST_DELAY = 2
TIMEOUT = 15  # Seconds before timing out a request
MAX_RETRIES = 2  # Retries per failed request
API_KEY = "cf4349d9377ef4a58c895a306e2cc50e6a011882e45e32dd07d0adce80b13a5b"
headers = {"X-API-Key": API_KEY}
LOCAL_SAVE_DIR = "./data/bronze/locations"
MAX_WORKERS = 4  # Parallel threads for location processing

os.makedirs(LOCAL_SAVE_DIR, exist_ok=True)

def save_local(data, location_id):
    """Save location data to local JSON file"""
    filename = f"location_{location_id}.json"
    filepath = os.path.join(LOCAL_SAVE_DIR, filename)
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2)
    return filepath

def fetch_with_retry(url, retries=MAX_RETRIES):
    """Fetch with exponential backoff retry"""
    for attempt in range(retries):
        try:
            response = requests.get(url, headers=headers, timeout=TIMEOUT)
            response.raise_for_status()
            return response
        except (requests.exceptions.RequestException, requests.exceptions.Timeout) as e:
            if attempt < retries - 1:
                wait = 2 ** attempt  # Exponential backoff
                print(f"Retrying in {wait}s... (Attempt {attempt + 1}/{retries})")
                time.sleep(wait)
                continue
            raise

def fetch_location_data(location_id):
    """Fetch data for a single location by ID"""
    url = f"{API_BASE}/locations/{location_id}"
    try:
        response = fetch_with_retry(url)
        data = response.json()
        if 'results' in data and data['results']:
            save_local(data, location_id)
            print(f"Saved location {location_id}")
        else:
            print(f"No data for location {location_id}")
    except Exception as e:
        print(f"Failed to fetch location {location_id}: {str(e)}")

def main():
    print(f"Saving data to: {os.path.abspath(LOCAL_SAVE_DIR)}")
    location_ids = range(1, 10001)

    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        futures = [executor.submit(fetch_location_data, location_id) for location_id in location_ids]

        for future in as_completed(futures):
            try:
                future.result()
            except Exception as e:
                print(f"Location processing failed: {str(e)}")

if __name__ == "__main__":
    start_time = datetime.now()
    try:
        main()
        duration = datetime.now() - start_time
        print(f"Completed in {duration}")
    except Exception as e:
        print(f"Script failed: {str(e)}")
        raise


Saving data to: c:\Users\joaob\OneDrive\Área de Trabalho\Efrei\Data Lakes\Projet\data\bronze\locations
Retrying in 1s... (Attempt 1/2)
Retrying in 1s... (Attempt 1/2)
Saved location 4
Saved location 3
Saved location 5Saved location 6

Saved location 8
Saved location 7
Saved location 9
Saved location 10
Failed to fetch location 1: 404 Client Error: Not Found for url: https://api.openaq.org/v3/locations/1
Failed to fetch location 2: 404 Client Error: Not Found for url: https://api.openaq.org/v3/locations/2
Saved location 12
Saved location 11
Saved location 14
Saved location 13
Saved location 16
Saved location 15
Saved location 17
Retrying in 1s... (Attempt 1/2)
Saved location 18
Saved location 19
Saved location 23Saved location 22

Saved location 21
Saved location 24Saved location 25

Saved location 26
Saved location 28
Saved location 27
Failed to fetch location 20: 404 Client Error: Not Found for url: https://api.openaq.org/v3/locations/20
Saved location 30
Saved location 31
Saved locat

In [3]:
pip install pyspark

     ---------------------------------------- 0.0/317.2 MB ? eta -:--:--
     ---------------------------------------- 0.0/317.2 MB ? eta -:--:--
     ---------------------------------------- 0.0/317.2 MB ? eta -:--:--
     ---------------------------------------- 0.0/317.2 MB ? eta -:--:--
     ---------------------------------------- 0.0/317.2 MB ? eta -:--:--
     ---------------------------------------- 0.0/317.2 MB ? eta -:--:--
     ---------------------------------------- 0.0/317.2 MB ? eta -:--:--
     ---------------------------------------- 0.3/317.2 MB ? eta -:--:--
     ---------------------------------------- 0.3/317.2 MB ? eta -:--:--
     -------------------------------------- 0.5/317.2 MB 598.5 kB/s eta 0:08:50
     -------------------------------------- 0.5/317.2 MB 598.5 kB/s eta 0:08:50
     -------------------------------------- 0.8/317.2 MB 633.2 kB/s eta 0:08:20
     -------------------------------------- 0.8/317.2 MB 633.2 kB/s eta 0:08:20
     ------------------


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Corrected bronze to silver

In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, ArrayType, BooleanType, LongType

def clean_location_data():
    spark = SparkSession.builder \
        .appName("RawToStaging_Locations") \
        .getOrCreate()

    # Define schema for the location data with corrected types
    location_schema = StructType([
        StructField("bounds", ArrayType(DoubleType())),
        StructField("coordinates", StructType([
            StructField("latitude", DoubleType()),
            StructField("longitude", DoubleType())
        ])),
        StructField("country", StructType([
            StructField("code", StringType()),
            StructField("id", LongType()),
            StructField("name", StringType())
        ])),
        StructField("datetimeFirst", StructType([
            StructField("local", StringType()),
            StructField("utc", StringType())
        ])),
        StructField("datetimeLast", StructType([
            StructField("local", StringType()),
            StructField("utc", StringType())
        ])),
        StructField("distance", StringType()),
        StructField("id", LongType()),
        StructField("instruments", ArrayType(StructType([
            StructField("id", LongType()),
            StructField("name", StringType())
        ]))),
        StructField("isMobile", BooleanType()),
        StructField("isMonitor", BooleanType()),
        StructField("licenses", ArrayType(StructType([
            StructField("attribution", StructType([
                StructField("name", StringType()),
                StructField("url", StringType())
            ])),
            StructField("dateFrom", StringType()),
            StructField("dateTo", StringType()),
            StructField("id", LongType()),
            StructField("name", StringType())
        ]))),
        StructField("locality", StringType()),
        StructField("name", StringType()),
        StructField("owner", StructType([
            StructField("id", LongType()),
            StructField("name", StringType())
        ])),
        StructField("provider", StructType([
            StructField("id", LongType()),
            StructField("name", StringType())
        ])),
        StructField("sensors", ArrayType(StructType([
            StructField("id", LongType()),
            StructField("name", StringType()),
            StructField("parameter", StructType([
                StructField("displayName", StringType()),
                StructField("id", LongType()),
                StructField("name", StringType()),
                StructField("units", StringType())
            ]))
        ]))),
        StructField("timezone", StringType())
    ])

    # Define the top-level schema
    schema = StructType([
        StructField("meta", StructType([
            StructField("found", LongType()),
            StructField("limit", LongType()),
            StructField("name", StringType()),
            StructField("page", LongType()),
            StructField("website", StringType())
        ])),
        StructField("results", ArrayType(location_schema))
    ])

    # Reading JSON with the updated schema
    raw_df = spark.read \
        .option("multiline", "true") \
        .schema(schema) \
        .json("data/bronze/locations/*.json")

    print(f"Raw data count: {raw_df.count()}")
    raw_df.show(5, truncate=False)

    # Explode the array into individual rows
    processed_df = raw_df.select(explode("results").alias("location")).select(
        col("location.id"),
        col("location.name"),
        col("location.locality"),
        col("location.timezone"),
        col("location.country.name").alias("country"),
        col("location.provider.name").alias("provider"),
        col("location.isMobile"),
        col("location.isMonitor"),
        col("location.coordinates.latitude").alias("lat"),
        col("location.coordinates.longitude").alias("lon")
    )

    print(f"Processed data count: {processed_df.count()}")
    processed_df.show(5, truncate=False)

    # Clean the data: remove duplicates and invalid rows
    cleaned_df = processed_df \
        .dropDuplicates(["id"]) \
        .na.drop(subset=["id", "name", "country", "lat", "lon"])

    print(f"Cleaned data count: {cleaned_df.count()}")
    cleaned_df.show(5, truncate=False)

    # Write the cleaned data to a local staging path
    cleaned_df.write \
        .mode("overwrite") \
        .parquet("data/silver/locations")

if __name__ == "__main__":
    clean_location_data()

Raw data count: 191
+--------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, when, count, avg, max, min

def transform_location_data():
    spark = SparkSession.builder \
        .appName("TransformLocationData") \
        .getOrCreate()

    # Read the cleaned data from the silver layer
    cleaned_df = spark.read.parquet("data/silver/locations")

    # Show the cleaned data for reference
    print("Cleaned Data:")
    cleaned_df.show(5, truncate=False)

    ### Transformation 1: Enrich Data with Country Code and Timezone
    # Add a new column `country_code` and `timezone` for easier analysis
    enriched_df = cleaned_df.withColumn("country_code", col("country")) \
                            .withColumn("timezone", col("timezone"))

    ### Transformation 2: Aggregate Data by Country
    # Count the number of locations per country and calculate average latitude/longitude
    country_stats_df = enriched_df.groupBy("country_code") \
        .agg(
            count("id").alias("location_count"),
            avg("lat").alias("avg_latitude"),
            avg("lon").alias("avg_longitude")
        )

    print("Country Statistics:")
    country_stats_df.show(5, truncate=False)

    ### Transformation 3: Identify Mobile and Monitor Locations
    # Filter and count mobile and monitor locations
    mobile_locations_df = enriched_df.filter(col("isMobile") == True) \
        .select("id", "name", "country_code", "lat", "lon")

    monitor_locations_df = enriched_df.filter(col("isMonitor") == True) \
        .select("id", "name", "country_code", "lat", "lon")

    print("Mobile Locations:")
    mobile_locations_df.show(5, truncate=False)

    print("Monitor Locations:")
    monitor_locations_df.show(5, truncate=False)

    ### Transformation 4: Calculate Geographic Bounds per Country
    # Find the min and max latitude/longitude for each country
    geographic_bounds_df = enriched_df.groupBy("country_code") \
        .agg(
            min("lat").alias("min_latitude"),
            max("lat").alias("max_latitude"),
            min("lon").alias("min_longitude"),
            max("lon").alias("max_longitude")
        )

    print("Geographic Bounds per Country:")
    geographic_bounds_df.show(5, truncate=False)

    ### Transformation 5: Create a Dataset for Provider Analysis
    # Count the number of locations per provider
    provider_stats_df = enriched_df.groupBy("provider") \
        .agg(
            count("id").alias("location_count")
        )

    print("Provider Statistics:")
    provider_stats_df.show(5, truncate=False)

    ### Write Transformed Data to Gold Layer
    # Write each transformed dataset to the gold layer
    country_stats_df.write \
        .mode("overwrite") \
        .parquet("data/gold/country_stats")

    mobile_locations_df.write \
        .mode("overwrite") \
        .parquet("data/gold/mobile_locations")

    monitor_locations_df.write \
        .mode("overwrite") \
        .parquet("data/gold/monitor_locations")

    geographic_bounds_df.write \
        .mode("overwrite") \
        .parquet("data/gold/geographic_bounds")

    provider_stats_df.write \
        .mode("overwrite") \
        .parquet("data/gold/provider_stats")

    print("Transformed data written to data/gold")

if __name__ == "__main__":
    transform_location_data()


Cleaned Data:
+---+-------------------+--------+------------+-------+----------------------------------+--------+---------+---------+----------+
|id |name               |locality|timezone    |country|provider                          |isMobile|isMonitor|lat      |lon       |
+---+-------------------+--------+------------+-------+----------------------------------+--------+---------+---------+----------+
|3  |NMA - Nima         |NULL    |Africa/Accra|Ghana  |Dr. Raphael E. Arku and Colleagues|false   |true     |5.58389  |-0.19968  |
|4  |NMT - Nima         |NULL    |Africa/Accra|Ghana  |Dr. Raphael E. Arku and Colleagues|false   |true     |5.58165  |-0.19898  |
|5  |JTA - Jamestown    |NULL    |Africa/Accra|Ghana  |Dr. Raphael E. Arku and Colleagues|false   |true     |5.5401139|-0.2103972|
|6  |ADT - Asylum Down  |NULL    |Africa/Accra|Ghana  |Dr. Raphael E. Arku and Colleagues|false   |true     |5.570722 |-0.2120555|
|7  |ADEPA - Asylum Down|NULL    |Africa/Accra|Ghana  |Dr. Raphael E.

In [12]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, count, avg

def analyze_sensor_instrument_data():
    spark = SparkSession.builder \
        .appName("AnalyzeSensorInstrumentData") \
        .getOrCreate()

    # Read the cleaned data from the silver layer
    cleaned_df = spark.read.parquet("data/silver/locations")

    # Show the cleaned data for reference
    print("Cleaned Data Schema:")
    cleaned_df.printSchema()

    print("Cleaned Data Sample:")
    cleaned_df.show(5, truncate=False)

    ### Check if the `sensors` column exists
    if "sensors" in cleaned_df.columns:
        ### Transformation 1: Explode Sensors
        exploded_sensors_df = cleaned_df.select(
            col("id").alias("location_id"),
            col("name").alias("location_name"),
            col("country").alias("country_name"),
            explode("sensors").alias("sensor")
        ).select(
            col("location_id"),
            col("location_name"),
            col("country_name"),
            col("sensor.id").alias("sensor_id"),
            col("sensor.name").alias("sensor_name"),
            col("sensor.parameter.displayName").alias("parameter_name"),
            col("sensor.parameter.units").alias("parameter_units")
        )

        print("Exploded Sensors Data:")
        exploded_sensors_df.show(5, truncate=False)

        ### Transformation 2: Sensor Statistics
        sensor_stats_df = exploded_sensors_df.groupBy("location_id", "location_name", "parameter_name") \
            .agg(
                count("sensor_id").alias("sensor_count")
            )

        print("Sensor Statistics:")
        sensor_stats_df.show(5, truncate=False)

        ### Transformation 3: Parameter Analysis
        parameter_stats_df = exploded_sensors_df.groupBy("parameter_name", "parameter_units") \
            .agg(
                count("sensor_id").alias("sensor_count")
            ).orderBy(col("sensor_count").desc())

        print("Parameter Statistics:")
        parameter_stats_df.show(5, truncate=False)

        ### Write Sensor Data to Gold Layer
        exploded_sensors_df.write \
            .mode("overwrite") \
            .parquet("data/gold/exploded_sensors")

        sensor_stats_df.write \
            .mode("overwrite") \
            .parquet("data/gold/sensor_stats")

        parameter_stats_df.write \
            .mode("overwrite") \
            .parquet("data/gold/parameter_stats")
    else:
        print("The `sensors` column is missing in the cleaned data. Skipping sensor-related transformations.")

    ### Check if the `instruments` column exists
    if "instruments" in cleaned_df.columns:
        ### Transformation 4: Explode Instruments
        exploded_instruments_df = cleaned_df.select(
            col("id").alias("location_id"),
            col("name").alias("location_name"),
            col("country").alias("country_name"),
            explode("instruments").alias("instrument")
        ).select(
            col("location_id"),
            col("location_name"),
            col("country_name"),
            col("instrument.id").alias("instrument_id"),
            col("instrument.name").alias("instrument_name")
        )

        print("Exploded Instruments Data:")
        exploded_instruments_df.show(5, truncate=False)

        ### Transformation 5: Instrument Statistics
        instrument_stats_df = exploded_instruments_df.groupBy("location_id", "location_name") \
            .agg(
                count("instrument_id").alias("instrument_count")
            )

        print("Instrument Statistics:")
        instrument_stats_df.show(5, truncate=False)

        ### Write Instrument Data to Gold Layer
        exploded_instruments_df.write \
            .mode("overwrite") \
            .parquet("data/gold/exploded_instruments")

        instrument_stats_df.write \
            .mode("overwrite") \
            .parquet("data/gold/instrument_stats")
    else:
        print("The `instruments` column is missing in the cleaned data. Skipping instrument-related transformations.")

    print("Transformed data written to data/gold")

if __name__ == "__main__":
    analyze_sensor_instrument_data()

Cleaned Data Schema:
root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- locality: string (nullable = true)
 |-- timezone: string (nullable = true)
 |-- country: string (nullable = true)
 |-- provider: string (nullable = true)
 |-- isMobile: boolean (nullable = true)
 |-- isMonitor: boolean (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)

Cleaned Data Sample:
+---+-------------------+--------+------------+-------+----------------------------------+--------+---------+---------+----------+
|id |name               |locality|timezone    |country|provider                          |isMobile|isMonitor|lat      |lon       |
+---+-------------------+--------+------------+-------+----------------------------------+--------+---------+---------+----------+
|3  |NMA - Nima         |NULL    |Africa/Accra|Ghana  |Dr. Raphael E. Arku and Colleagues|false   |true     |5.58389  |-0.19968  |
|4  |NMT - Nima         |NULL    |Africa/Accra|G

# Test with AWS S3

In [4]:
!pip install python-dotenv


  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import requests
import json
import os
import time
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
import boto3  # AWS SDK for Python
from dotenv import load_dotenv

# Configuration
API_BASE = "https://api.openaq.org/v3"
REQUEST_DELAY = 2
TIMEOUT = 15  # Seconds before timing out a request
MAX_RETRIES = 2  # Retries per failed request
API_KEY = "cf4349d9377ef4a58c895a306e2cc50e6a011882e45e32dd07d0adce80b13a5b"
headers = {"X-API-Key": API_KEY}
S3_BUCKET_NAME = "openaq-locations-data"  # Replace with your S3 bucket name
S3_SAVE_DIR = "bronze/locations"  # S3 folder path
MAX_WORKERS = 4  # Parallel threads for location processing

load_dotenv()

# Initialize S3 client
s3_client = boto3.client(
    "s3",
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    region_name=os.getenv("AWS_DEFAULT_REGION"),
)

def save_to_s3(data, location_id):
    """Save location data to S3 bucket"""
    filename = f"location_{location_id}.json"
    s3_key = f"{S3_SAVE_DIR}/{filename}"
    try:
        s3_client.put_object(
            Bucket=S3_BUCKET_NAME,
            Key=s3_key,
            Body=json.dumps(data, indent=2),
            ContentType="application/json",
        )
        print(f"Saved location {location_id} to S3: s3://{S3_BUCKET_NAME}/{s3_key}")
    except Exception as e:
        print(f"Failed to save location {location_id} to S3: {str(e)}")

def fetch_with_retry(url, retries=MAX_RETRIES):
    """Fetch with exponential backoff retry"""
    for attempt in range(retries):
        try:
            response = requests.get(url, headers=headers, timeout=TIMEOUT)
            response.raise_for_status()
            return response
        except (requests.exceptions.RequestException, requests.exceptions.Timeout) as e:
            if attempt < retries - 1:
                wait = 2 ** attempt  # Exponential backoff
                print(f"Retrying in {wait}s... (Attempt {attempt + 1}/{retries})")
                time.sleep(wait)
                continue
            raise

def fetch_location_data(location_id):
    """Fetch data for a single location by ID"""
    url = f"{API_BASE}/locations/{location_id}"
    try:
        response = fetch_with_retry(url)
        data = response.json()
        if "results" in data and data["results"]:
            save_to_s3(data, location_id)
        else:
            print(f"No data for location {location_id}")
    except Exception as e:
        print(f"Failed to fetch location {location_id}: {str(e)}")

def main():
    print(f"Saving data to S3 bucket: {S3_BUCKET_NAME}/{S3_SAVE_DIR}")
    location_ids = range(1, 10001)

    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        futures = [executor.submit(fetch_location_data, location_id) for location_id in location_ids]

        for future in as_completed(futures):
            try:
                future.result()
            except Exception as e:
                print(f"Location processing failed: {str(e)}")

if __name__ == "__main__":
    start_time = datetime.now()
    try:
        main()
        duration = datetime.now() - start_time
        print(f"Completed in {duration}")
    except Exception as e:
        print(f"Script failed: {str(e)}")
        raise

Saving data to S3 bucket: openaq-locations-data/bronze/locations
Retrying in 1s... (Attempt 1/2)
Retrying in 1s... (Attempt 1/2)
Failed to fetch location 2: 404 Client Error: Not Found for url: https://api.openaq.org/v3/locations/2
Failed to fetch location 1: 404 Client Error: Not Found for url: https://api.openaq.org/v3/locations/1
Saved location 4 to S3: s3://openaq-locations-data/bronze/locations/location_4.jsonSaved location 3 to S3: s3://openaq-locations-data/bronze/locations/location_3.json

Saved location 5 to S3: s3://openaq-locations-data/bronze/locations/location_5.json
Saved location 6 to S3: s3://openaq-locations-data/bronze/locations/location_6.json
Saved location 8 to S3: s3://openaq-locations-data/bronze/locations/location_8.json
Saved location 7 to S3: s3://openaq-locations-data/bronze/locations/location_7.json
Saved location 9 to S3: s3://openaq-locations-data/bronze/locations/location_9.json
Saved location 10 to S3: s3://openaq-locations-data/bronze/locations/location

In [2]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, ArrayType, BooleanType, LongType
from dotenv import load_dotenv


def clean_location_data():
    # Load environment variables
    load_dotenv()
    
    # Validate credentials
    aws_access_key = os.getenv("AWS_ACCESS_KEY_ID")
    aws_secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")
    
    if not aws_access_key or not aws_secret_key:
        raise ValueError("AWS credentials not found in environment variables!")
    
    # Initialize Spark with S3 configurations
    spark = SparkSession.builder \
        .appName("RawToStaging_Locations") \
        .config("spark.hadoop.fs.s3a.access.key", aws_access_key) \
        .config("spark.hadoop.fs.s3a.secret.key", aws_secret_key) \
        .config("spark.hadoop.fs.s3a.endpoint", "s3.amazonaws.com") \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.1,com.amazonaws:aws-java-sdk-bundle:1.12.262") \
        .getOrCreate()

    # Define schema for the location data with corrected types
    location_schema = StructType([
        StructField("bounds", ArrayType(DoubleType())),
        StructField("coordinates", StructType([
            StructField("latitude", DoubleType()),
            StructField("longitude", DoubleType())
        ])),
        StructField("country", StructType([
            StructField("code", StringType()),
            StructField("id", LongType()),
            StructField("name", StringType())
        ])),
        StructField("datetimeFirst", StructType([
            StructField("local", StringType()),
            StructField("utc", StringType())
        ])),
        StructField("datetimeLast", StructType([
            StructField("local", StringType()),
            StructField("utc", StringType())
        ])),
        StructField("distance", StringType()),
        StructField("id", LongType()),
        StructField("instruments", ArrayType(StructType([
            StructField("id", LongType()),
            StructField("name", StringType())
        ]))),
        StructField("isMobile", BooleanType()),
        StructField("isMonitor", BooleanType()),
        StructField("licenses", ArrayType(StructType([
            StructField("attribution", StructType([
                StructField("name", StringType()),
                StructField("url", StringType())
            ])),
            StructField("dateFrom", StringType()),
            StructField("dateTo", StringType()),
            StructField("id", LongType()),
            StructField("name", StringType())
        ]))),
        StructField("locality", StringType()),
        StructField("name", StringType()),
        StructField("owner", StructType([
            StructField("id", LongType()),
            StructField("name", StringType())
        ])),
        StructField("provider", StructType([
            StructField("id", LongType()),
            StructField("name", StringType())
        ])),
        StructField("sensors", ArrayType(StructType([
            StructField("id", LongType()),
            StructField("name", StringType()),
            StructField("parameter", StructType([
                StructField("displayName", StringType()),
                StructField("id", LongType()),
                StructField("name", StringType()),
                StructField("units", StringType())
            ]))
        ]))),
        StructField("timezone", StringType())
    ])

    # Define the top-level schema
    schema = StructType([
        StructField("meta", StructType([
            StructField("found", LongType()),
            StructField("limit", LongType()),
            StructField("name", StringType()),
            StructField("page", LongType()),
            StructField("website", StringType())
        ])),
        StructField("results", ArrayType(location_schema))
    ])

    # S3 paths
    bronze_path = "s3a://openaq-locations-data/bronze/locations/*.json"  # Replace with your S3 bronze path
    silver_path = "s3a://openaq-locations-data/silver/locations"  # Replace with your S3 silver path

    # Reading JSON with the updated schema from S3
    raw_df = spark.read \
        .option("multiline", "true") \
        .schema(schema) \
        .json(bronze_path)

    print(f"Raw data count: {raw_df.count()}")
    raw_df.show(5, truncate=False)

    # Explode the array into individual rows
    processed_df = raw_df.select(explode("results").alias("location")).select(
        col("location.id"),
        col("location.name"),
        col("location.locality"),
        col("location.timezone"),
        col("location.country.name").alias("country"),
        col("location.provider.name").alias("provider"),
        col("location.isMobile"),
        col("location.isMonitor"),
        col("location.coordinates.latitude").alias("lat"),
        col("location.coordinates.longitude").alias("lon")
    )

    print(f"Processed data count: {processed_df.count()}")
    processed_df.show(5, truncate=False)

    # Clean the data: remove duplicates and invalid rows
    cleaned_df = processed_df \
        .dropDuplicates(["id"]) \
        .na.drop(subset=["id", "name", "country", "lat", "lon"])

    print(f"Cleaned data count: {cleaned_df.count()}")
    cleaned_df.show(5, truncate=False)

    # Write the cleaned data to S3
    cleaned_df.write \
        .mode("overwrite") \
        .parquet(silver_path)

    print(f"Cleaned data written to S3: {silver_path}")

if __name__ == "__main__":
        
    clean_location_data()

Raw data count: 78
+--------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [3]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, avg, max, min
from dotenv import load_dotenv

def transform_location_data():
    # Load environment variables
    load_dotenv()

    # Validate credentials
    aws_access_key = os.getenv("AWS_ACCESS_KEY_ID")
    aws_secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")

    if not aws_access_key or not aws_secret_key:
        raise ValueError("AWS credentials not found in environment variables!")

    # Initialize Spark session with S3 configurations
    spark = SparkSession.builder \
        .appName("TransformLocationData") \
        .config("spark.hadoop.fs.s3a.access.key", aws_access_key) \
        .config("spark.hadoop.fs.s3a.secret.key", aws_secret_key) \
        .config("spark.hadoop.fs.s3a.endpoint", "s3.amazonaws.com") \
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.1,com.amazonaws:aws-java-sdk-bundle:1.12.262") \
        .getOrCreate()

    # S3 paths
    silver_path = "s3a://openaq-locations-data/silver/locations"  # Replace with your S3 silver path
    gold_path = "s3a://openaq-locations-data/gold"  # Replace with your S3 gold path

    # Read the cleaned data from the silver layer
    cleaned_df = spark.read.parquet(silver_path)

    # Show the cleaned data for reference
    print("Cleaned Data:")
    cleaned_df.show(5, truncate=False)

    ### Transformation 1: Enrich Data with Country Code and Timezone
    # Add a new column `country_code` and `timezone` for easier analysis
    enriched_df = cleaned_df.withColumn("country_code", col("country")) \
                            .withColumn("timezone", col("timezone"))

    ### Transformation 2: Aggregate Data by Country
    # Count the number of locations per country and calculate average latitude/longitude
    country_stats_df = enriched_df.groupBy("country_code") \
        .agg(
            count("id").alias("location_count"),
            avg("lat").alias("avg_latitude"),
            avg("lon").alias("avg_longitude")
        )

    print("Country Statistics:")
    country_stats_df.show(5, truncate=False)

    ### Transformation 3: Identify Mobile and Monitor Locations
    # Filter and count mobile and monitor locations
    mobile_locations_df = enriched_df.filter(col("isMobile") == True) \
        .select("id", "name", "country_code", "lat", "lon")

    monitor_locations_df = enriched_df.filter(col("isMonitor") == True) \
        .select("id", "name", "country_code", "lat", "lon")

    print("Mobile Locations:")
    mobile_locations_df.show(5, truncate=False)

    print("Monitor Locations:")
    monitor_locations_df.show(5, truncate=False)

    ### Transformation 4: Calculate Geographic Bounds per Country
    # Find the min and max latitude/longitude for each country
    geographic_bounds_df = enriched_df.groupBy("country_code") \
        .agg(
            min("lat").alias("min_latitude"),
            max("lat").alias("max_latitude"),
            min("lon").alias("min_longitude"),
            max("lon").alias("max_longitude")
        )

    print("Geographic Bounds per Country:")
    geographic_bounds_df.show(5, truncate=False)

    ### Transformation 5: Create a Dataset for Provider Analysis
    # Count the number of locations per provider
    provider_stats_df = enriched_df.groupBy("provider") \
        .agg(
            count("id").alias("location_count")
        )

    print("Provider Statistics:")
    provider_stats_df.show(5, truncate=False)

    ### Write Transformed Data to Gold Layer
    # Write each transformed dataset to the gold layer in S3
    country_stats_df.write \
        .mode("overwrite") \
        .parquet(f"{gold_path}/country_stats")

    mobile_locations_df.write \
        .mode("overwrite") \
        .parquet(f"{gold_path}/mobile_locations")

    monitor_locations_df.write \
        .mode("overwrite") \
        .parquet(f"{gold_path}/monitor_locations")

    geographic_bounds_df.write \
        .mode("overwrite") \
        .parquet(f"{gold_path}/geographic_bounds")

    provider_stats_df.write \
        .mode("overwrite") \
        .parquet(f"{gold_path}/provider_stats")

    print(f"Transformed data written to S3: {gold_path}")

if __name__ == "__main__":
    transform_location_data()

Cleaned Data:
+---+-------------------+--------+------------+-------+----------------------------------+--------+---------+---------+----------+
|id |name               |locality|timezone    |country|provider                          |isMobile|isMonitor|lat      |lon       |
+---+-------------------+--------+------------+-------+----------------------------------+--------+---------+---------+----------+
|3  |NMA - Nima         |NULL    |Africa/Accra|Ghana  |Dr. Raphael E. Arku and Colleagues|false   |true     |5.58389  |-0.19968  |
|4  |NMT - Nima         |NULL    |Africa/Accra|Ghana  |Dr. Raphael E. Arku and Colleagues|false   |true     |5.58165  |-0.19898  |
|5  |JTA - Jamestown    |NULL    |Africa/Accra|Ghana  |Dr. Raphael E. Arku and Colleagues|false   |true     |5.5401139|-0.2103972|
|6  |ADT - Asylum Down  |NULL    |Africa/Accra|Ghana  |Dr. Raphael E. Arku and Colleagues|false   |true     |5.570722 |-0.2120555|
|7  |ADEPA - Asylum Down|NULL    |Africa/Accra|Ghana  |Dr. Raphael E.

In [4]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, count
from dotenv import load_dotenv

def analyze_sensor_instrument_data():
    # Load environment variables
    load_dotenv()

    # Validate credentials
    aws_access_key = os.getenv("AWS_ACCESS_KEY_ID")
    aws_secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")

    if not aws_access_key or not aws_secret_key:
        raise ValueError("AWS credentials not found in environment variables!")

    # Initialize Spark session with S3 configurations
    spark = SparkSession.builder \
        .appName("AnalyzeSensorInstrumentData") \
        .config("spark.hadoop.fs.s3a.access.key", aws_access_key) \
        .config("spark.hadoop.fs.s3a.secret.key", aws_secret_key) \
        .config("spark.hadoop.fs.s3a.endpoint", "s3.amazonaws.com") \
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.1,com.amazonaws:aws-java-sdk-bundle:1.12.262") \
        .getOrCreate()

    # S3 paths
    silver_path = "s3a://openaq-locations-data/silver/locations"  # Replace with your S3 silver path
    gold_path = "s3a://openaq-locations-data/gold"  # Replace with your S3 gold path

    # Read the cleaned data from the silver layer
    cleaned_df = spark.read.parquet(silver_path)

    # Show the cleaned data for reference
    print("Cleaned Data Schema:")
    cleaned_df.printSchema()

    print("Cleaned Data Sample:")
    cleaned_df.show(5, truncate=False)

    ### Check if the `sensors` column exists
    if "sensors" in cleaned_df.columns:
        ### Transformation 1: Explode Sensors
        exploded_sensors_df = cleaned_df.select(
            col("id").alias("location_id"),
            col("name").alias("location_name"),
            col("country").alias("country_name"),
            explode("sensors").alias("sensor")
        ).select(
            col("location_id"),
            col("location_name"),
            col("country_name"),
            col("sensor.id").alias("sensor_id"),
            col("sensor.name").alias("sensor_name"),
            col("sensor.parameter.displayName").alias("parameter_name"),
            col("sensor.parameter.units").alias("parameter_units")
        )

        print("Exploded Sensors Data:")
        exploded_sensors_df.show(5, truncate=False)

        ### Transformation 2: Sensor Statistics
        sensor_stats_df = exploded_sensors_df.groupBy("location_id", "location_name", "parameter_name") \
            .agg(
                count("sensor_id").alias("sensor_count")
            )

        print("Sensor Statistics:")
        sensor_stats_df.show(5, truncate=False)

        ### Transformation 3: Parameter Analysis
        parameter_stats_df = exploded_sensors_df.groupBy("parameter_name", "parameter_units") \
            .agg(
                count("sensor_id").alias("sensor_count")
            ).orderBy(col("sensor_count").desc())

        print("Parameter Statistics:")
        parameter_stats_df.show(5, truncate=False)

        ### Write Sensor Data to Gold Layer
        exploded_sensors_df.write \
            .mode("overwrite") \
            .parquet(f"{gold_path}/exploded_sensors")

        sensor_stats_df.write \
            .mode("overwrite") \
            .parquet(f"{gold_path}/sensor_stats")

        parameter_stats_df.write \
            .mode("overwrite") \
            .parquet(f"{gold_path}/parameter_stats")
    else:
        print("The `sensors` column is missing in the cleaned data. Skipping sensor-related transformations.")

    ### Check if the `instruments` column exists
    if "instruments" in cleaned_df.columns:
        ### Transformation 4: Explode Instruments
        exploded_instruments_df = cleaned_df.select(
            col("id").alias("location_id"),
            col("name").alias("location_name"),
            col("country").alias("country_name"),
            explode("instruments").alias("instrument")
        ).select(
            col("location_id"),
            col("location_name"),
            col("country_name"),
            col("instrument.id").alias("instrument_id"),
            col("instrument.name").alias("instrument_name")
        )

        print("Exploded Instruments Data:")
        exploded_instruments_df.show(5, truncate=False)

        ### Transformation 5: Instrument Statistics
        instrument_stats_df = exploded_instruments_df.groupBy("location_id", "location_name") \
            .agg(
                count("instrument_id").alias("instrument_count")
            )

        print("Instrument Statistics:")
        instrument_stats_df.show(5, truncate=False)

        ### Write Instrument Data to Gold Layer
        exploded_instruments_df.write \
            .mode("overwrite") \
            .parquet(f"{gold_path}/exploded_instruments")

        instrument_stats_df.write \
            .mode("overwrite") \
            .parquet(f"{gold_path}/instrument_stats")
    else:
        print("The `instruments` column is missing in the cleaned data. Skipping instrument-related transformations.")

    print(f"Transformed data written to S3: {gold_path}")

if __name__ == "__main__":
    analyze_sensor_instrument_data()

Cleaned Data Schema:
root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- locality: string (nullable = true)
 |-- timezone: string (nullable = true)
 |-- country: string (nullable = true)
 |-- provider: string (nullable = true)
 |-- isMobile: boolean (nullable = true)
 |-- isMonitor: boolean (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)

Cleaned Data Sample:
+---+-------------------+--------+------------+-------+----------------------------------+--------+---------+---------+----------+
|id |name               |locality|timezone    |country|provider                          |isMobile|isMonitor|lat      |lon       |
+---+-------------------+--------+------------+-------+----------------------------------+--------+---------+---------+----------+
|3  |NMA - Nima         |NULL    |Africa/Accra|Ghana  |Dr. Raphael E. Arku and Colleagues|false   |true     |5.58389  |-0.19968  |
|4  |NMT - Nima         |NULL    |Africa/Accra|G